# Overview

In the 10x series of notebooks, we will look at Time Series modeling in pycaret using univariate data and no exogenous variables. We will use the famous airline dataset for illustration. Our plan of action is as follows:

1. Perform EDA on the dataset to extract valuable insight about the process generating the time series. **(Covered in this notebook)**
2. Model the dataset based on exploratory analysis (univariable model without exogenous variables).
3. Use an automated approach (AutoML) to improve the performance.

In [1]:
# Only enable critical logging (Optional)
import os
os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"

In [2]:
def what_is_installed():
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except ModuleNotFoundError:
    !pip install pycaret
    what_is_installed()


System:
    python: 3.10.11 (v3.10.11:7d4cc5aa85, Apr  4 2023, 19:05:19) [Clang 13.0.0 (clang-1300.0.29.30)]
executable: /usr/local/bin/python3
   machine: macOS-15.1-arm64-arm-64bit

PyCaret required dependencies:
                 pip: 25.3
          setuptools: 80.9.0
             pycaret: 3.3.2
             IPython: 8.37.0
          ipywidgets: 8.1.8
                tqdm: 4.67.1
               numpy: 1.26.4
              pandas: 2.1.4
              jinja2: 3.1.6
               scipy: 1.11.4
              joblib: 1.3.2
             sklearn: 1.4.2
                pyod: 2.0.5
            imblearn: 0.14.0
   category_encoders: 2.7.0
            lightgbm: 4.6.0
               numba: 0.61.0
            requests: 2.32.5
          matplotlib: 3.7.5
          scikitplot: 0.3.7
         yellowbrick: 1.5
              plotly: 5.24.1
    plotly-resampler: Not installed
             kaleido: 1.1.0
           schemdraw: 0.15
         statsmodels: 0.14.5
              sktime: 0.26.0
             

In [3]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment

In [4]:
# Global Figure Settings for notebook ----
global_fig_settings = {
    "renderer": "notebook",  # Changed from "png" to "notebook" for interactive display
    "width": 1000,
    "height": 600,
    "template": "plotly"  # Add template for better visualization
}

In [5]:
y = get_data('airline', verbose=False)

In [6]:
# We want to forecast the next 12 months of data and we will use 3 fold cross-validation to test the models.
fh = 12 # or alternately fh = np.arange(1,13)
fold = 3

In [7]:
# Global Figure Settings for notebook ----
# Depending on whether you are using jupyter notebook, jupyter lab, Google Colab, you may have to set the renderer appropriately
# NOTE: Setting to a static renderer here so that the notebook saved size is reduced.
fig_kwargs = {
    # "renderer": "notebook",
    "renderer": "png",
    "width": 1000,
    "height": 600,
}

# Exploratory Analysis

`pycaret` Time Series Forecasting module provides a convenient interface for perform exploratory analysis using `plot_model`.

**NOTE:**
* Without an estimator argument, `plot_model` will plot using the original dataset. We will cover this in the current notebook.
* If an estimator (model) is passed to `plot_model`, the the plots are made using the model data (e.g. future forecasts, or analysis on insample residuals). We will cover this in a subsequent notebook.

Let's see how this works next.

**First, we will plots the original dataset.**

In [8]:
eda = TSForecastingExperiment()
eda.setup(data=y, fh=fh, fig_kwargs=fig_kwargs)

,Description,Value
0,session_id,8696
1,Target,Number of airline passengers
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(144, 1)"
5,Transformed data shape,"(144, 1)"
6,Transformed train set shape,"(132, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [9]:
# NOTE: This is the same as `eda.plot_model(plot="ts")`
eda.plot_model(
    fig_kwargs={
        "renderer": "notebook",
        "width": 1000,
        "height": 1200,
        "template": "plotly"
    },
    display_format="plotly-widget"  # Enable interactive display
)

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'color': 'rgb(31, 119, 180)', 'size': 5},
              'mode': 'lines+markers',
              'name': 'Number of airline passengers (original)',
              'type': 'scattergl',
              'uid': 'a9227d18-533c-4405-9f07-c21e11315d36',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
            

Before we explore the data further, there are a few minor things to know about how PyCaret prepares a **modeling pipeline** under the hood. The data being modeled is usually fed through an internal pipeline that has optional steps in the following order:

**Data Input (by user) >> Imputation >> Transformation & Scaling >> Model**

1. **Imputation**
  - This step is optional if data does not have missing values, and is mandatory if data has missing values. This is because many statistical tests and models can not work with missing data.
  - Although some models like `Prophet` can work with missing data, the need to run statistical tests to extract useful information from the data for default model settings necessitates having imputation when data has missing values.
  

2. **Transformations and Scaling**
  - This step is optional and user should usually only enable this after evaluating the models (e.g. by performing residual analysis), or if they have specific requirements such as limiting the forecast to only positive values.
  
  
We will discuss more about imputation and transformations in in another notebook. For now, our data does not have any missing values or any transformations. So **Data Input (by user), i.e. Original data = Imputed data = Transformed data = Data fed to models**. We can verify this by plotting the internal datasets by specifying the `plot_data_type` `data_kwargs`.

NOTE: If `plot_data_type` is not provided, each plot type has it's own default data_type that is internally determined, but the users have the ability to always overwrite using `plot_data_type`.

In [10]:
eda.plot_model(
    fig_kwargs={
        "renderer": "notebook",
        "width": 1000,
        "height": 1200,
        "template": "plotly"
    },
    display_format="plotly-widget",  # Enable interactive display
    data_kwargs={"plot_data_type": ["original", "imputed", "transformed"]})

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'color': 'rgb(31, 119, 180)', 'size': 5},
              'mode': 'lines+markers',
              'name': 'Number of airline passengers (original)',
              'type': 'scattergl',
              'uid': '2bccade9-3dee-4b5b-b908-ab2c5e2ca586',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
            

**Let's explore the standard ACF and PACF plots for the dataset next**

In [11]:
# ACF and PACF for the original dataset
eda.plot_model(
    display_format="plotly-widget",
    plot="acf")

FigureWidgetResampler({
    'data': [{'line': {'color': '#3f3f3f'},
              'marker': {'size': 10},
              'mode': 'lines',
              'name': 'ACF',
              'showlegend': False,
              'type': 'scattergl',
              'uid': 'f1fafcde-9683-49ba-947c-67687f20efb7',
              'x': array([0, 0]),
              'xaxis': 'x',
              'y': array([0., 1.]),
              'yaxis': 'y'},
             {'line': {'color': '#3f3f3f'},
              'marker': {'size': 10},
              'mode': 'lines',
              'name': 'ACF',
              'showlegend': False,
              'type': 'scattergl',
              'uid': 'c9533ac4-b8be-465a-a893-92b388f6433f',
              'x': array([1, 1]),
              'xaxis': 'x',
              'y': array([0.        , 0.94804734]),
              'yaxis': 'y'},
             {'line': {'color': '#3f3f3f'},
              'marker': {'size': 10},
              'mode': 'lines',
              'name': 'ACF',
              'sho

In [12]:
# NOTE: you can customize the plots with kwargs - e.g. number of lags, figure size (width, height), etc
# data_kwargs such as `nlags` are passed to the underlying function that gets the ACF values
# figure kwargs such as `fig_size` & `template` are passed to plotly and can have any value that plotly accepts
eda.plot_model(plot="pacf", data_kwargs={'nlags':36}, fig_kwargs={'height': 500, "width": 800}, display_format="plotly-widget")

FigureWidgetResampler({
    'data': [{'line': {'color': '#3f3f3f'},
              'marker': {'size': 10},
              'mode': 'lines',
              'name': 'PACF',
              'showlegend': False,
              'type': 'scattergl',
              'uid': 'e3299364-cf31-41a8-b921-6ebe38ea0233',
              'x': array([0, 0]),
              'xaxis': 'x',
              'y': array([0., 1.]),
              'yaxis': 'y'},
             {'line': {'color': '#3f3f3f'},
              'marker': {'size': 10},
              'mode': 'lines',
              'name': 'PACF',
              'showlegend': False,
              'type': 'scattergl',
              'uid': 'fd7668f9-2490-4f4d-866f-a63cc985b7ed',
              'x': array([1, 1]),
              'xaxis': 'x',
              'y': array([0.        , 0.95467704]),
              'yaxis': 'y'},
             {'line': {'color': '#3f3f3f'},
              'marker': {'size': 10},
              'mode': 'lines',
              'name': 'PACF',
              '

**Users may also wish to explore the spectrogram or the FFT which are very useful for studying the frequency components in the time series.**

For example:
- Peaking at f =~ 0 can indicating wandering behavior characteristic of a random walk that needs to be differenced. This could also be indicative of a stationary ARMA process with a high positive phi value.
- Peaking at a frequency and its multiples is indicative of seasonality. The lowest frequency in this case is called the fundamental frequency and the inverse of this frequency is the seasonal period for the model.

In [13]:
eda.plot_model(display_format="plotly-widget", plot="periodogram")
eda.plot_model(display_format="plotly-widget", plot="fft")

FigureWidgetResampler({
    'data': [{'hoverinfo': 'text',
              'hovertext': array(['Freq: 0.0069<br>Ampl: 53.3557<br>Time period: 144.0',
                                  'Freq: 0.0139<br>Ampl: 49.6072<br>Time period: 72.0',
                                  'Freq: 0.0208<br>Ampl: 39.1849<br>Time period: 48.0',
                                  'Freq: 0.0278<br>Ampl: 32.1925<br>Time period: 36.0',
                                  'Freq: 0.0347<br>Ampl: 28.4396<br>Time period: 28.8',
                                  'Freq: 0.0417<br>Ampl: 28.2206<br>Time period: 24.0',
                                  'Freq: 0.0486<br>Ampl: 12.8238<br>Time period: 20.5714',
                                  'Freq: 0.0556<br>Ampl: 26.128<br>Time period: 18.0',
                                  'Freq: 0.0625<br>Ampl: 22.4787<br>Time period: 16.0',
                                  'Freq: 0.0694<br>Ampl: 34.0128<br>Time period: 14.4',
                                  'Freq: 0.0764<br>Ampl: 4

FigureWidgetResampler({
    'data': [{'hoverinfo': 'text',
              'hovertext': array(['Freq: 0.0139<br>Ampl: 72.2535<br>Time period: 72.0',
                                  'Freq: 0.0208<br>Ampl: 70.283<br>Time period: 48.0',
                                  'Freq: 0.0278<br>Ampl: 67.3361<br>Time period: 36.0',
                                  'Freq: 0.0347<br>Ampl: 66.1099<br>Time period: 28.8',
                                  'Freq: 0.0417<br>Ampl: 62.2454<br>Time period: 24.0',
                                  'Freq: 0.0486<br>Ampl: 62.0267<br>Time period: 20.5714',
                                  'Freq: 0.0556<br>Ampl: 62.8348<br>Time period: 18.0',
                                  'Freq: 0.0625<br>Ampl: 60.9445<br>Time period: 16.0',
                                  'Freq: 0.0694<br>Ampl: 61.9179<br>Time period: 14.4',
                                  'Freq: 0.0764<br>Ampl: 65.2939<br>Time period: 13.0909',
                                  'Freq: 0.0833<br>Ampl:

**In the plots above, we notice**

1. Peaking at f ~= 0 indicating that we need to difference the data.
2. Peaking at f = 0.0833, 0.1677, 0.25, 0.3333, 0.4167. All these are multiple of 0.0833. Hence 0.0833 is the fundamental frequency and the seasonal period is 1/0.0833 = 12.

**Alternately, the `diagnostics` plot provides all these details in a convenient call.**

In [14]:
eda.plot_model(plot="diagnostics", display_format="plotly-widget", fig_kwargs={"height": 800, "width": 1000})

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'color': 'rgb(31, 119, 180)', 'size': 5},
              'mode': 'lines+markers',
              'name': 'Number of airline passengers (transformed)',
              'showlegend': False,
              'type': 'scattergl',
              'uid': '5f96059c-ad3f-4c72-90de-1dd05dfc5474',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.dat

Our diagnosutic plots indicated the need to difference and the presence of a seasonal period of 12. **Let's see what happens when we remove this from the model. What other characteristics are left in the model that would need to be taken care of?**

This can be achieved through the difference plots. Along with the difference plots, we will plot the corresponding ACF, PACF and Periodogram for further diagnostics.

In [15]:
# Row 1: Original
# Row 2: d = 1
# Row 3: First (d = 1) then (D = 1, s = 12)
#   - Corresponds to applying a standard first difference to handle trend, and
#     followed by a seasonal difference (at lag 12) to attempt to account for
#     seasonal dependence.
# Ref: https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.transformations.series.difference.Differencer.html
eda.plot_model(
    plot="diff",
    display_format="plotly-widget",
    data_kwargs={"lags_list": [[1], [1, 12]], "acf": True, "pacf": True, "periodogram": True},
    fig_kwargs={"height": 800, "width": 1500}
)


# ## NOTE: Another way to specify differences is using order_list
# # Row 1: Original
# # Row 2: d = 1
# # Row 3: d = 2
# eda.plot_model(
#     plot="diff",
#     data_kwargs={
#         "order_list": [1, 2],
#         "acf": True, "pacf": True, "periodogram": True
#     },
#     fig_kwargs={"height": 600, "width": 1200}
# )

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'color': 'rgb(31, 119, 180)', 'size': 5},
              'mode': 'lines+markers',
              'name': 'Actual',
              'showlegend': False,
              'type': 'scattergl',
              'uid': '4d32a428-86b1-4d32-8e41-5ab3fede596e',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
          

**Observations:**

1. In the second row, we have only removed the wandering behavior by taking a first difference. This can be seen in the ACF plot (extended autocorrelations are gone) and Periodogram (peaking at f =~ 0 is squished). The ACF (preaking at seasonal period of 12 and its multiples) and PACF (peaking at fundamental frequency of 0.0833 and its multiples) still show the seasonal behavior.
2. In the third row, we have taken first difference followed by a seasonal difference of 12. Now, we can see that the peaking at seasonal multiples is gone from both ACF and Periodogram. There is still a little bit of autoregresssive properties that we have not taken care of but by looking at the PACF, maybe p=1 seems like a reasonable value to use (most lags after that are insignificant).

**Conclusion**
* If you were modeling this with ARIMA, the model to try would be **ARIMA(1,1,0)x(0,1,0,12)**.
* Other models could use this information appropriately. For example, reduced regression models could remove the trend and seasonality of 12 (i.e. make the data stationary) before modeling the rest of the autoregressive properties. Luckily, the `pycaret` time series module will take care of this internally.


**Let's plot the Time Series Decomposition next (another classical diagnostic plot)**

In [16]:
# First, classical decomposition
# By default the seasonal period is the one detected during setup - 12 in this case.
eda.plot_model(plot="decomp", display_format="plotly-widget", fig_kwargs={"height": 500})

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'size': 2},
              'mode': 'lines+markers',
              'name': 'Actual',
              'type': 'scattergl',
              'uid': '50c05a11-1822-42cb-affe-b95828e54d47',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.datetime(1949, 11, 1, 0, 0),
           

In [17]:
# Users can change the seasonal period to explore what is best for this model.
eda.plot_model(plot="decomp", data_kwargs={'seasonal_period': 24}, display_format="plotly-widget", fig_kwargs={"height": 500})

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'size': 2},
              'mode': 'lines+markers',
              'name': 'Actual',
              'type': 'scattergl',
              'uid': 'd4182cc9-9f1c-4407-a0d3-82c073bab2ea',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.datetime(1949, 11, 1, 0, 0),
           

In [18]:
# Users may wish to customize the decomposition, for example, in this case multiplicative seasonality
# probably makes more sense since the magnitide of the seasonality increase as the time progresses
eda.plot_model(plot="decomp", data_kwargs={'type': 'multiplicative'}, display_format="plotly-widget", fig_kwargs={"height": 500})

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'size': 2},
              'mode': 'lines+markers',
              'name': 'Actual',
              'type': 'scattergl',
              'uid': '521bc358-af9e-4f5c-988f-ce6a8111f913',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.datetime(1949, 11, 1, 0, 0),
           

In [19]:
# Users can also plot STL decomposition
# Reference: https://otexts.com/fpp2/stl.html
eda.plot_model(plot="decomp_stl", display_format="plotly-widget", fig_kwargs={"height": 500})

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'size': 2},
              'mode': 'lines+markers',
              'name': 'Actual',
              'type': 'scattergl',
              'uid': 'b617ade5-7aae-4af9-80c0-37114df5e588',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.datetime(1949, 11, 1, 0, 0),
           

**Let us look at the various splits of the data used for modeling next.**

**NOTE:**
* In time series, we can not split the data randomly since there is serial correlation in the data and using future data to predict past data will result in leakage. Hence the temporal dependence must be maintained when splitting the data.
* Users may wish to refer to this for more details:
  - https://github.com/pycaret/pycaret/discussions/1761
  - https://robjhyndman.com/hyndsight/tscv/
  - https://topepo.github.io/caret/data-splitting.html#data-splitting-for-time-series

In [20]:
# Show the train-test splits on the dataset
# Internally split - len(fh) as test set, remaining used as test set
eda.plot_model(plot="train_test_split", display_format="plotly-widget", fig_kwargs={"height": 400, "width": 900})

# Show the Cross Validation splits inside the train set
# The blue dots represent the training data for each fold.
# The orange dots represent the validation data for each fold
eda.plot_model(plot="cv", display_format="plotly-widget", fig_kwargs={"height": 400, "width": 900})

FigureWidgetResampler({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'width': 2},
              'marker': {'color': 'rgb(31, 119, 180)', 'size': 5},
              'mode': 'lines+markers',
              'name': 'Train (original)',
              'type': 'scattergl',
              'uid': '36f34fd6-3ba3-4a83-95c3-eef368c2c8ef',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.

FigureWidgetResampler({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': '#C0C0C0'},
              'mode': 'lines+markers',
              'name': 'Unchanged',
              'showlegend': False,
              'type': 'scattergl',
              'uid': 'c29d323c-74d6-4925-bc93-69396b5c6b83',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0)], dtype=object),
              'y': array([2, 2])},
             {'hoverinfo': 'skip',
              'line': {'color': '#C0C0C0'},
              'mode': 'lines+markers',
              'name': 'Unchanged',
              'showlegend': False,
              'type': 'scattergl',
              'uid': '1d7629e1-56ae-444e-9fc6-fd32d455470a',
              'x': array([datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0)], dtype=object),
              'y': array([2, 2])},
             {'hoverinfo': 'skip',
              'line

# Statistical Tests

Statistical Testing is another important part of time series modeling. This can be achieved easily in pycaret using `check_stats`.

**Options are:**
* 'summary',
* 'white_noise'
* 'stationarity'
* 'adf'
* 'kpss'
* 'normality'
* 'all'

By default the statistics are run on the "transformed" data, but similar to plots, the user has the abiliy to set this to another data type using the `data_type` argument. e.g. `eda.check_stats(test="summary", data_type = "original")`

In [21]:
# Summary Statistics
eda.check_stats(test="summary")

,Test,Test Name,Data,Property,Setting,Value
0,Summary,Statistics,Transformed,Length,,144.000000
1,Summary,Statistics,Transformed,# Missing Values,,0.000000
2,Summary,Statistics,Transformed,Mean,,280.298611
3,Summary,Statistics,Transformed,Median,,265.500000
4,Summary,Statistics,Transformed,Standard Deviation,,119.966317
5,Summary,Statistics,Transformed,Variance,,14391.917201
6,Summary,Statistics,Transformed,Kurtosis,,-0.364942
7,Summary,Statistics,Transformed,Skewness,,0.583160
8,Summary,Statistics,Transformed,# Distinct Values,,118.000000


In [22]:
# Stationarity tests (ADF and KPSS)
# NOTE: Users can also just run a single test by passing either 'adf' or 'kpss' to `check_stats`
eda.check_stats(test='stationarity')

,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,ADF,Transformed,Stationarity,{'alpha': 0.05},False
1,Stationarity,ADF,Transformed,p-value,{'alpha': 0.05},0.99188
2,Stationarity,ADF,Transformed,Test Statistic,{'alpha': 0.05},0.815369
3,Stationarity,ADF,Transformed,Critical Value 1%,{'alpha': 0.05},-3.481682
4,Stationarity,ADF,Transformed,Critical Value 5%,{'alpha': 0.05},-2.884042
5,Stationarity,ADF,Transformed,Critical Value 10%,{'alpha': 0.05},-2.57877
6,Stationarity,KPSS,Transformed,Trend Stationarity,{'alpha': 0.05},True
7,Stationarity,KPSS,Transformed,p-value,{'alpha': 0.05},0.1
8,Stationarity,KPSS,Transformed,Test Statistic,{'alpha': 0.05},0.09615
9,Stationarity,KPSS,Transformed,Critical Value 10%,{'alpha': 0.05},0.119


The ADF tests shows that the data is not stationary and we saw this in the plots as well (clear trend and seasonal behavior)

In [23]:
# Ljung-Bx test to tests of white noise (whether the data is uncorrelated or not)
eda.check_stats(test='white_noise')

,Test,Test Name,Data,Property,Setting,Value
0,White Noise,Ljung-Box,Transformed,Test Statictic,"{'alpha': 0.05, 'K': 24}",1606.083817
1,White Noise,Ljung-Box,Transformed,Test Statictic,"{'alpha': 0.05, 'K': 48}",1933.155822
2,White Noise,Ljung-Box,Transformed,p-value,"{'alpha': 0.05, 'K': 24}",0.000000
3,White Noise,Ljung-Box,Transformed,p-value,"{'alpha': 0.05, 'K': 48}",0.000000
4,White Noise,Ljung-Box,Transformed,White Noise,"{'alpha': 0.05, 'K': 24}",0.000000
5,White Noise,Ljung-Box,Transformed,White Noise,"{'alpha': 0.05, 'K': 48}",0.000000


The Ljung-Box tests indicates that the data is not white noise - again something that was clearly visible in the data

In [24]:
# Users have the option to customize the tests such as change the alpha value.
eda.check_stats(test='kpss', alpha = 0.2)

,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,KPSS,Transformed,Trend Stationarity,{'alpha': 0.2},False
1,Stationarity,KPSS,Transformed,p-value,{'alpha': 0.2},0.1
2,Stationarity,KPSS,Transformed,Test Statistic,{'alpha': 0.2},0.09615
3,Stationarity,KPSS,Transformed,Critical Value 10%,{'alpha': 0.2},0.119
4,Stationarity,KPSS,Transformed,Critical Value 5%,{'alpha': 0.2},0.146
5,Stationarity,KPSS,Transformed,Critical Value 2.5%,{'alpha': 0.2},0.176
6,Stationarity,KPSS,Transformed,Critical Value 1%,{'alpha': 0.2},0.216


Alternately, all the above tests can be done in one shot by not passing any test type.

In [25]:
eda.check_stats()

,Test,Test Name,Data,Property,Setting,Value
0,Summary,Statistics,Transformed,Length,,144.0
1,Summary,Statistics,Transformed,# Missing Values,,0.0
2,Summary,Statistics,Transformed,Mean,,280.298611
3,Summary,Statistics,Transformed,Median,,265.5
4,Summary,Statistics,Transformed,Standard Deviation,,119.966317
5,Summary,Statistics,Transformed,Variance,,14391.917201
6,Summary,Statistics,Transformed,Kurtosis,,-0.364942
7,Summary,Statistics,Transformed,Skewness,,0.58316
8,Summary,Statistics,Transformed,# Distinct Values,,118.0
9,White Noise,Ljung-Box,Transformed,Test Statictic,"{'alpha': 0.05, 'K': 24}",1606.083817


**That's it for this notebook. In the next notebook, we will see how we can start to model this data.**